In [ ]:
#librerias
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import math
import numpy as np

import missingno as msno 
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold

# Import necessary modules
from sklearn.metrics import mean_squared_error

# Keras specific
import keras
from keras.models import Sequential
from keras.layers import Dense

from tensorflow.keras import layers
from keras.layers.recurrent import LSTM

from keras.models import Sequential
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.core import Activation
from keras.layers.core import Dropout
from keras.layers.core import Dense
from keras.layers import Flatten
from keras.layers import Input
from keras.models import Model
from keras.callbacks import ModelCheckpoint
from tensorflow.keras import layers
from keras.callbacks import History,EarlyStopping
from tensorflow.keras.regularizers import L1, L2, L1L2

from sklearn.metrics import classification_report

import tensorflow as tf
#import tensorflow_addons as tfa
from tensorflow import keras
import pydot



#help(mp_pose.Pose)


from numpy.random import seed
import random as python_random
import random

import warnings
warnings.filterwarnings("ignore")

seed(48)
python_random.seed(48)
tf.random.set_seed(48)

#Cambiar el formato de las fechas a dd-mm-aa
import datetime
import plotly.graph_objects as go
from plotly.offline import download_plotlyjs,init_notebook_mode,plot,iplot
from sklearn.metrics import mean_squared_error

from keras.layers import LSTM

from datetime import timedelta

from keras.models import Sequential, Model, load_model
from keras.utils.vis_utils import plot_model, model_to_dot
from keras.layers import LSTM, Dense, RepeatVector, TimeDistributed, Input, BatchNormalization, multiply, concatenate, Flatten, Activation, dot

from keras.optimizers import Adam
#from attention_decoder import AttentionDecoder

In [ ]:
# hacer la conexión a Google Drive

from google.colab import drive

colab_flag = True

if (colab_flag):
    drive.mount('/content/drive',force_remount=True)

path = '/content/drive/MyDrive/ANN-2021-I/'

Mounted at /content/drive


# Preprocesamiento

In [ ]:
train= pd.read_csv(path+'velocidad-viento/windspeed-training.csv')
train

,Fecha,Velocidad de viento en 20.0 metros [mean]
0,2012-12-19 00:10,12.4
1,2012-12-19 00:20,11.4
2,2012-12-19 00:30,11.8
3,2012-12-19 00:40,11.6
4,2012-12-19 00:50,11.0
...,...,...
123359,2015-04-24 23:10,0.5
123360,2015-04-24 23:20,0.7
123361,2015-04-24 23:30,1.3
123362,2015-04-24 23:40,2.6


In [ ]:
test= pd.read_csv(path+'velocidad-viento/windspeed-testing.csv')
test

,26-04-2015 01:50
0,26-04-2015 03:10
1,26-04-2015 04:20
2,26-04-2015 05:20
3,26-04-2015 06:40
4,26-04-2015 10:30
...,...
1994,19-09-2015 08:10
1995,21-09-2015 11:20
1996,22-09-2015 10:30
1997,23-09-2015 17:30


In [ ]:
train.columns= ["Fecha","Velocidad"]
train["Fecha"]= train.Fecha.apply(lambda x: datetime.datetime.strptime(x, '%Y-%m-%d %H:%M'))

In [ ]:
def preprocesamiento(data, n_input, n_output, n_step):

  for i in range(0, data.shape[0], n_step):
    try:
      x = data[i:i+n_input].reshape(1,n_input)
      y = data[i+n_input: i+n_input+n_output].reshape(1,n_output)
      
      if i == 0:
        X = x
        Y = y
      else:
        X = np.vstack((X,x))
        Y = np.vstack((Y,y))
    except:
      pass

  return X.reshape(X.shape[0], X.shape[1], 1), Y

**Idea**: si el comportamiento es periodico en los meses, se entrenará la red para que aprenda a predecir mes a mes, a partir de comportamiento del mes en años anteriores

In [ ]:
abril_2013 = train[(train['Fecha']<= '2013-05-01')&('2013-03-27' <= train['Fecha'])]
abril_2014 = train[(train['Fecha']<= '2014-05-01')&('2014-03-27' <= train['Fecha'])]

j = 0
data = []
Traces = {
1 :'trace0',
2: 'trace1',
3: 'trace2'
}

Traces[0] = go.Scatter(
         x = train['Fecha'] ,
         y = train['Velocidad'],
         mode = 'lines',
         name = "Train"
     )
data.append(Traces[j]) 

Traces[1] = go.Scatter(
         x = abril_2014['Fecha'] ,
         y = abril_2013['Velocidad'],
         mode = 'lines',
         name = "abril_2013"
     )
data.append(Traces[1])


layout = go.Layout(
      xaxis=dict(title='Fechas'),
      yaxis=dict(title='Velocidad'),
      title=('Velocidad...'))
fig = go.Figure(data=data, layout=layout)
iplot(fig)

#LSTM Model

In [ ]:
def model(X):
  regressor = Sequential()

  # Adding the first LSTM layer and some Dropout regularisation
  regressor.add(LSTM(units = 50, return_sequences = True, input_shape = (X.shape[1], 1)))
  regressor.add(Dropout(0.2))

  # Adding a second LSTM layer and some Dropout regularisation
  regressor.add(LSTM(units = 50, return_sequences = True))
  regressor.add(Dropout(0.2))

  # Adding a third LSTM layer and some Dropout regularisation
  regressor.add(LSTM(units = 50, return_sequences = True))
  regressor.add(Dropout(0.2))

  # Adding a fourth LSTM layer and some Dropout regularisation
  regressor.add(LSTM(units = 50))
  regressor.add(Dropout(0.2))

  # Adding the output layer
  regressor.add(Dense(units = 1))

  # Compiling the RNN
  regressor.compile(optimizer = 'adam', loss = 'mean_squared_error')

  return regressor

In [ ]:
def invertir(fecha):
  l = fecha.split(' ')
  l2 = l[0].split('-')

  return l2[2]+ '-' + l2[1] + '-' + l2[0] + ' ' + l[1]

In [ ]:
def df_predicciones(df, y_pred):

  pred = pd.DataFrame(columns=['Fecha','Velocidad'])

  pred['Fecha'] = list(df['Fecha']+ timedelta(days=365))[3:]
  pred['Velocidad'] = y_pred

  return pred

In [ ]:
def predicciones(fecha_inf_2013, fecha_sup_2013, fecha_inf_2014, fecha_sup_2014):

  df_2013 = train[(train['Fecha']<= fecha_sup_2013)&(fecha_inf_2013 <= train['Fecha'])]
  df_2014 = train[(train['Fecha']<= fecha_sup_2014)&(fecha_inf_2014 <= train['Fecha'])]

  X, _ = preprocesamiento(np.array(df_2013.Velocidad), 3, 1, 1)
  _, y = preprocesamiento(np.array(df_2014.Velocidad), 3, 1, 1)
  regressor = model(X)

  regressor.fit(X, y, epochs = 50, verbose=1, batch_size = 32)

  X2, y2 = preprocesamiento(np.array(df_2014.Velocidad), 3, 1, 1)

  y_pred = regressor.predict(X2)

  pred = df_predicciones(df_2014, y_pred)

  return pred

In [ ]:
pred_abril_2015 = predicciones('2013-03-27', '2013-05-01', '2014-03-27', '2014-05-01')

Epoch 1/50
158/158 [==============================] - 48s 10ms/step - loss: 44.4787
Epoch 2/50
158/158 [==============================] - 1s 9ms/step - loss: 17.8760
Epoch 3/50
158/158 [==============================] - 2s 10ms/step - loss: 17.4327
Epoch 4/50
158/158 [==============================] - 2s 10ms/step - loss: 17.0532
Epoch 5/50
158/158 [==============================] - 2s 10ms/step - loss: 16.3830
Epoch 6/50
158/158 [==============================] - 2s 10ms/step - loss: 16.6298
Epoch 7/50
158/158 [==============================] - 2s 10ms/step - loss: 16.9306
Epoch 8/50
158/158 [==============================] - 1s 9ms/step - loss: 16.6240
Epoch 9/50
158/158 [==============================] - 1s 9ms/step - loss: 16.3708
Epoch 10/50
158/158 [==============================] - 1s 9ms/step - loss: 17.1810
Epoch 11/50
158/158 [==============================] - 2s 11ms/step - loss: 17.0462
Epoch 12/50
158/158 [==============================] - 2s 10ms/step - loss: 16.2197
Epoc

In [ ]:
pred_mayo_2015 = predicciones('2013-04-27', '2013-06-01', '2014-04-27', '2014-06-01')

Epoch 1/50
158/158 [==============================] - 8s 9ms/step - loss: 43.3879
Epoch 2/50
158/158 [==============================] - 1s 9ms/step - loss: 16.4777
Epoch 3/50
158/158 [==============================] - 2s 10ms/step - loss: 17.1811
Epoch 4/50
158/158 [==============================] - 2s 9ms/step - loss: 17.5752
Epoch 5/50
158/158 [==============================] - 2s 10ms/step - loss: 16.9952
Epoch 6/50
158/158 [==============================] - 2s 10ms/step - loss: 16.5935
Epoch 7/50
158/158 [==============================] - 2s 10ms/step - loss: 17.0699
Epoch 8/50
158/158 [==============================] - 2s 10ms/step - loss: 16.5365
Epoch 9/50
158/158 [==============================] - 2s 11ms/step - loss: 16.7957
Epoch 10/50
158/158 [==============================] - 2s 10ms/step - loss: 16.6728
Epoch 11/50
158/158 [==============================] - 1s 9ms/step - loss: 16.9719
Epoch 12/50
158/158 [==============================] - 2s 10ms/step - loss: 16.2959
Epoch

In [ ]:
pred_junio_2015 = predicciones('2013-05-27', '2013-07-01', '2014-05-27', '2014-07-01')

Epoch 1/50
158/158 [==============================] - 8s 10ms/step - loss: 39.7725
Epoch 2/50
158/158 [==============================] - 2s 10ms/step - loss: 14.9611
Epoch 3/50
158/158 [==============================] - 2s 11ms/step - loss: 15.0325
Epoch 4/50
158/158 [==============================] - 2s 10ms/step - loss: 15.0979
Epoch 5/50
158/158 [==============================] - 1s 9ms/step - loss: 14.9522
Epoch 6/50
158/158 [==============================] - 2s 10ms/step - loss: 15.1532
Epoch 7/50
158/158 [==============================] - 2s 10ms/step - loss: 14.9759
Epoch 8/50
158/158 [==============================] - 2s 10ms/step - loss: 15.2554
Epoch 9/50
158/158 [==============================] - 2s 11ms/step - loss: 15.2879
Epoch 10/50
158/158 [==============================] - 2s 11ms/step - loss: 14.8569
Epoch 11/50
158/158 [==============================] - 2s 11ms/step - loss: 14.9330
Epoch 12/50
158/158 [==============================] - 2s 10ms/step - loss: 14.8940
Ep

In [ ]:
pred_julio_2015 = predicciones('2013-06-27', '2013-08-01', '2014-06-27', '2014-08-01')

Epoch 1/50
158/158 [==============================] - 8s 9ms/step - loss: 50.7381
Epoch 2/50
158/158 [==============================] - 1s 9ms/step - loss: 19.4752
Epoch 3/50
158/158 [==============================] - 1s 9ms/step - loss: 19.9173
Epoch 4/50
158/158 [==============================] - 2s 10ms/step - loss: 20.2539
Epoch 5/50
158/158 [==============================] - 2s 10ms/step - loss: 19.3914
Epoch 6/50
158/158 [==============================] - 2s 11ms/step - loss: 19.5295
Epoch 7/50
158/158 [==============================] - 2s 10ms/step - loss: 19.7301
Epoch 8/50
158/158 [==============================] - 2s 10ms/step - loss: 19.5107
Epoch 9/50
158/158 [==============================] - 2s 10ms/step - loss: 19.7999
Epoch 10/50
158/158 [==============================] - 2s 10ms/step - loss: 19.4814
Epoch 11/50
158/158 [==============================] - 2s 10ms/step - loss: 19.4901
Epoch 12/50
158/158 [==============================] - 1s 9ms/step - loss: 19.6890
Epoch

In [ ]:
pred_agosto_2015 = predicciones('2013-07-27', '2013-09-01', '2014-07-27', '2014-09-01')

Epoch 1/50
162/162 [==============================] - 8s 9ms/step - loss: 37.8035
Epoch 2/50
162/162 [==============================] - 2s 9ms/step - loss: 16.3251
Epoch 3/50
162/162 [==============================] - 2s 10ms/step - loss: 15.9135
Epoch 4/50
162/162 [==============================] - 2s 10ms/step - loss: 16.0988
Epoch 5/50
162/162 [==============================] - 2s 10ms/step - loss: 16.0922
Epoch 6/50
162/162 [==============================] - 2s 10ms/step - loss: 15.7047
Epoch 7/50
162/162 [==============================] - 2s 10ms/step - loss: 16.1251
Epoch 8/50
162/162 [==============================] - 1s 9ms/step - loss: 16.1062
Epoch 9/50
162/162 [==============================] - 1s 9ms/step - loss: 15.5462
Epoch 10/50
162/162 [==============================] - 1s 9ms/step - loss: 16.1987
Epoch 11/50
162/162 [==============================] - 2s 9ms/step - loss: 15.3818
Epoch 12/50
162/162 [==============================] - 2s 9ms/step - loss: 16.0030
Epoch 13

In [ ]:
pred_septiembre_2015 = predicciones('2013-08-27', '2013-10-01', '2014-08-27', '2014-10-01')

Epoch 1/50
158/158 [==============================] - 8s 9ms/step - loss: 49.4098
Epoch 2/50
158/158 [==============================] - 2s 10ms/step - loss: 17.2643
Epoch 3/50
158/158 [==============================] - 1s 9ms/step - loss: 17.5908
Epoch 4/50
158/158 [==============================] - 1s 9ms/step - loss: 16.9952
Epoch 5/50
158/158 [==============================] - 1s 9ms/step - loss: 17.0401
Epoch 6/50
158/158 [==============================] - 1s 9ms/step - loss: 16.6501
Epoch 7/50
158/158 [==============================] - 1s 9ms/step - loss: 15.9576
Epoch 8/50
158/158 [==============================] - 1s 9ms/step - loss: 15.6715
Epoch 9/50
158/158 [==============================] - 1s 9ms/step - loss: 14.9936
Epoch 10/50
158/158 [==============================] - 2s 10ms/step - loss: 15.5383
Epoch 11/50
158/158 [==============================] - 1s 9ms/step - loss: 14.9601
Epoch 12/50
158/158 [==============================] - 2s 10ms/step - loss: 15.0892
Epoch 13/5

In [ ]:
predicciones = pd.concat([pred_abril_2015,pred_mayo_2015, pred_junio_2015, pred_julio_2015, pred_agosto_2015, pred_septiembre_2015], axis=0)
predicciones.reset_index(drop=True)

,Fecha,Velocidad
0,2015-03-27 00:30:00,8.315536
1,2015-03-27 00:40:00,8.384362
2,2015-03-27 00:50:00,8.296227
3,2015-03-27 01:00:00,7.485259
4,2015-03-27 01:10:00,6.754833
...,...,...
30367,2015-09-30 23:20:00,8.772756
30368,2015-09-30 23:30:00,8.270269
30369,2015-09-30 23:40:00,8.184086
30370,2015-09-30 23:50:00,8.854185


In [ ]:
l = [test.columns[0]]
l = l+ [i[0] for i in test.values]

test_2 = pd.DataFrame(columns=['Fechas'])

test_2['Fechas'] = l
test_2

entregable = pd.DataFrame(columns=['Fecha', 'Velocidad'])

j=0

for i in test_2['Fechas']:
  entregable.loc[j] = [i, predicciones[predicciones['Fecha']==invertir(i)]['Velocidad'].values[0]]
  j+=1

In [ ]:
def vector(listatupla):
    summit=pd.DataFrame(listatupla)
    summit.columns=["'Fecha'","'Velocidad de viento en 20.0 metros [mean]'"]
    summit=summit.set_index("'Fecha'")
    return summit

In [ ]:
entregable = vector(entregable)
entregable.to_csv('sampleSubmission.csv')

# Endocer - Decoder

In [ ]:
def df_predicciones_luong(df, y_pred):

  n,m = y_pred.shape[0], y_pred.shape[1]

  pred = pd.DataFrame(columns=['Fecha','Velocidad'])

  pred['Fecha'] = list(df['Fecha']+ timedelta(days=365))[2:-1]
  pred['Velocidad'] = y_pred.reshape(n*m)

  return pred

In [ ]:
def encoder_decoder(train_x, train_y):

	n_timesteps, n_features, n_outputs = train_x.shape[1], train_x.shape[2], train_y.shape[1]
  
  # define model
	model = Sequential()
	model.add(LSTM(200, activation='relu', input_shape=(n_timesteps, n_features)))
	model.add(RepeatVector(n_outputs))
	model.add(LSTM(200, activation='relu', return_sequences=True))
	model.add(TimeDistributed(Dense(100, activation='relu')))
	model.add(TimeDistributed(Dense(1)))
	model.compile(loss='mse', optimizer='adam')
	return model

In [ ]:
def encoder_decoder_2(train_x, train_y):
  model = Sequential()
	
  n_timesteps, n_features, n_outputs = train_x.shape[1], train_x.shape[2], train_y.shape[1]

  #Adding the first LSTM layer and some Dropout regularisation
  model.add(LSTM(units = 50, return_sequences = True, input_shape = (n_timesteps, n_features)))
  model.add(Dropout(0.2))

  # Adding a second LSTM layer and some Dropout regularisation
  model.add(LSTM(units = 50, return_sequences = True))
  model.add(Dropout(0.2))

  # Adding a third LSTM layer and some Dropout regularisation
  model.add(LSTM(units = 50, return_sequences = True))
  model.add(Dropout(0.2))

  # Adding a fourth LSTM layer and some Dropout regularisation
  model.add(LSTM(units = 50))
  model.add(Dropout(0.2))
  model.add(RepeatVector(n_outputs))
  model.add(LSTM(100, activation='relu', return_sequences=True))
  model.add(TimeDistributed(Dense(100, activation='relu')))
  model.add(TimeDistributed(Dense(1)))
  model.compile(loss='mse', optimizer='adam')
 
 
  return model

In [ ]:
def predicciones_encoder_decoder(fecha_inf_2013, fecha_sup_2013, fecha_inf_2014, fecha_sup_2014):

  df_2013 = train[(train['Fecha']<= fecha_sup_2013)&(fecha_inf_2013 <= train['Fecha'])]
  df_2014 = train[(train['Fecha']<= fecha_sup_2014)&(fecha_inf_2014 <= train['Fecha'])]

  X, _ = preprocesamiento(np.array(df_2013.Velocidad), 3, 2, 2)
  _, y = preprocesamiento(np.array(df_2014.Velocidad), 3, 2, 2)
  
  model = encoder_decoder_2(X,y)

  model.fit(X, y, epochs = 10, verbose=1, batch_size = 32)

  X2, y2 = preprocesamiento(np.array(df_2014.Velocidad), 3, 2, 2)

  y_pred = model.predict(X2)

  pred = df_predicciones_luong(df_2014, y_pred)

  return pred

In [ ]:
df_2013 = train[(train['Fecha']<= '2013-05-01')&('2013-03-27' <= train['Fecha'])]
df_2014 = train[(train['Fecha']<= '2014-05-01')&('2014-03-27' <= train['Fecha'])]

X, _ = preprocesamiento(np.array(df_2013.Velocidad), 3, 2, 2)
_, y = preprocesamiento(np.array(df_2014.Velocidad), 3, 2, 2)

In [ ]:
X_train, X_val = X[:-1], X[-1]
y_train, y_val = y[:-1], y[-1]

In [ ]:
X_val.shape

(3, 1)

In [ ]:
from sklearn.metrics import mean_squared_error

In [ ]:
model = encoder_decoder_2(X_train,y_train)

model.fit(X_train, y_train, epochs = 10, verbose=1, batch_size = 128)

#X2, y2 = preprocesamiento(np.array(df_2014.Velocidad), 3, 2, 2)

y_pred = model.predict(X_val.reshape(1, X_val.shape[0], X_val.shape[1]))

Epoch 1/10
20/20 [==============================] - 8s 20ms/step - loss: 75.8888
Epoch 2/10
20/20 [==============================] - 0s 20ms/step - loss: 50.2048
Epoch 3/10
20/20 [==============================] - 0s 21ms/step - loss: 27.9957
Epoch 4/10
20/20 [==============================] - 0s 19ms/step - loss: 21.7284
Epoch 5/10
20/20 [==============================] - 0s 20ms/step - loss: 17.3988
Epoch 6/10
20/20 [==============================] - 0s 20ms/step - loss: 17.6656
Epoch 7/10
20/20 [==============================] - 0s 19ms/step - loss: 17.6943
Epoch 8/10
20/20 [==============================] - 0s 19ms/step - loss: 18.1426
Epoch 9/10
20/20 [==============================] - 0s 20ms/step - loss: 17.8400
Epoch 10/10
20/20 [==============================] - 0s 20ms/step - loss: 17.9843


In [ ]:
pred_abril_2015 = predicciones_encoder_decoder('2013-03-27', '2013-05-01', '2014-03-27', '2014-05-01')

Epoch 1/10
79/79 [==============================] - 31s 12ms/step - loss: 61.1204
Epoch 2/10
79/79 [==============================] - 1s 11ms/step - loss: 17.6805
Epoch 3/10
79/79 [==============================] - 1s 12ms/step - loss: 18.0152
Epoch 4/10
79/79 [==============================] - 1s 13ms/step - loss: 17.5975
Epoch 5/10
79/79 [==============================] - 1s 12ms/step - loss: 17.1535
Epoch 6/10
79/79 [==============================] - 1s 12ms/step - loss: 17.0662
Epoch 7/10
79/79 [==============================] - 1s 12ms/step - loss: 16.9203
Epoch 8/10
79/79 [==============================] - 1s 11ms/step - loss: 17.5092
Epoch 9/10
79/79 [==============================] - 1s 12ms/step - loss: 17.1337
Epoch 10/10
79/79 [==============================] - 1s 13ms/step - loss: 17.1754


In [ ]:
pred_mayo_2015 = predicciones_encoder_decoder('2013-04-27', '2013-06-01', '2014-04-27', '2014-06-01')

Epoch 1/10
79/79 [==============================] - 9s 11ms/step - loss: 64.0262
Epoch 2/10
79/79 [==============================] - 1s 12ms/step - loss: 18.1581
Epoch 3/10
79/79 [==============================] - 1s 13ms/step - loss: 17.3568
Epoch 4/10
79/79 [==============================] - 1s 11ms/step - loss: 17.1653
Epoch 5/10
79/79 [==============================] - 1s 11ms/step - loss: 17.1004
Epoch 6/10
79/79 [==============================] - 1s 13ms/step - loss: 17.0241
Epoch 7/10
79/79 [==============================] - 1s 12ms/step - loss: 17.2905
Epoch 8/10
79/79 [==============================] - 1s 13ms/step - loss: 17.2003
Epoch 9/10
79/79 [==============================] - 1s 11ms/step - loss: 16.7663
Epoch 10/10
79/79 [==============================] - 1s 12ms/step - loss: 17.3691


In [ ]:
pred_junio_2015 = predicciones_encoder_decoder('2013-05-27', '2013-07-01', '2014-05-27', '2014-07-01')

Epoch 1/10
79/79 [==============================] - 9s 13ms/step - loss: 56.9952
Epoch 2/10
79/79 [==============================] - 1s 11ms/step - loss: 16.6491
Epoch 3/10
79/79 [==============================] - 1s 11ms/step - loss: 15.9234
Epoch 4/10
79/79 [==============================] - 1s 13ms/step - loss: 15.2448
Epoch 5/10
79/79 [==============================] - 1s 12ms/step - loss: 14.9654
Epoch 6/10
79/79 [==============================] - 1s 11ms/step - loss: 15.0162
Epoch 7/10
79/79 [==============================] - 1s 12ms/step - loss: 14.5675
Epoch 8/10
79/79 [==============================] - 1s 12ms/step - loss: 15.0076
Epoch 9/10
79/79 [==============================] - 1s 12ms/step - loss: 14.9293
Epoch 10/10
79/79 [==============================] - 1s 12ms/step - loss: 14.8020


In [ ]:
pred_julio_2015 = predicciones_encoder_decoder('2013-06-27', '2013-08-01', '2014-06-27', '2014-08-01')

Epoch 1/10
79/79 [==============================] - 9s 11ms/step - loss: 68.5389
Epoch 2/10
79/79 [==============================] - 1s 12ms/step - loss: 19.7260
Epoch 3/10
79/79 [==============================] - 1s 11ms/step - loss: 19.2648
Epoch 4/10
79/79 [==============================] - 1s 12ms/step - loss: 19.5570
Epoch 5/10
79/79 [==============================] - 1s 13ms/step - loss: 20.3140
Epoch 6/10
79/79 [==============================] - 1s 11ms/step - loss: 19.4159
Epoch 7/10
79/79 [==============================] - 1s 12ms/step - loss: 19.8626
Epoch 8/10
79/79 [==============================] - 1s 13ms/step - loss: 19.2963
Epoch 9/10
79/79 [==============================] - 1s 13ms/step - loss: 19.7283
Epoch 10/10
79/79 [==============================] - 1s 13ms/step - loss: 19.9896


In [ ]:
pred_agosto_2015 = predicciones_encoder_decoder('2013-07-27', '2013-09-01', '2014-07-27', '2014-09-01')

Epoch 1/10
81/81 [==============================] - 10s 11ms/step - loss: 52.7458
Epoch 2/10
81/81 [==============================] - 1s 11ms/step - loss: 16.8948
Epoch 3/10
81/81 [==============================] - 1s 11ms/step - loss: 16.4753
Epoch 4/10
81/81 [==============================] - 1s 13ms/step - loss: 16.3793
Epoch 5/10
81/81 [==============================] - 1s 12ms/step - loss: 16.2855
Epoch 6/10
81/81 [==============================] - 1s 12ms/step - loss: 15.4082
Epoch 7/10
81/81 [==============================] - 1s 11ms/step - loss: 15.3198
Epoch 8/10
81/81 [==============================] - 1s 13ms/step - loss: 16.3398
Epoch 9/10
81/81 [==============================] - 1s 13ms/step - loss: 14.6290
Epoch 10/10
81/81 [==============================] - 1s 12ms/step - loss: 15.7543


In [ ]:
pred_septiembre_2015 = predicciones_encoder_decoder('2013-08-27', '2013-10-01', '2014-08-27', '2014-10-01')

Epoch 1/10
79/79 [==============================] - 9s 12ms/step - loss: 71.4298
Epoch 2/10
79/79 [==============================] - 1s 13ms/step - loss: 16.9617
Epoch 3/10
79/79 [==============================] - 1s 12ms/step - loss: 17.8873
Epoch 4/10
79/79 [==============================] - 1s 11ms/step - loss: 17.5685
Epoch 5/10
79/79 [==============================] - 1s 12ms/step - loss: 16.7628
Epoch 6/10
79/79 [==============================] - 1s 12ms/step - loss: 15.9592
Epoch 7/10
79/79 [==============================] - 1s 12ms/step - loss: 16.2243
Epoch 8/10
79/79 [==============================] - 1s 11ms/step - loss: 15.6500
Epoch 9/10
79/79 [==============================] - 1s 12ms/step - loss: 15.9061
Epoch 10/10
79/79 [==============================] - 1s 12ms/step - loss: 14.9319


In [ ]:
predicciones = pd.concat([pred_abril_2015,pred_mayo_2015, pred_junio_2015, pred_julio_2015, pred_agosto_2015, pred_septiembre_2015], axis=0)
predicciones.reset_index(drop=True)

,Fecha,Velocidad
0,2015-03-27 00:20:00,7.851914
1,2015-03-27 00:30:00,7.815901
2,2015-03-27 00:40:00,8.072370
3,2015-03-27 00:50:00,8.012380
4,2015-03-27 01:00:00,7.361711
...,...,...
30367,2015-09-30 23:10:00,8.078589
30368,2015-09-30 23:20:00,7.746674
30369,2015-09-30 23:30:00,7.954499
30370,2015-09-30 23:40:00,7.608668


In [ ]:
l = [test.columns[0]]
l = l+ [i[0] for i in test.values]

test_2 = pd.DataFrame(columns=['Fechas'])

test_2['Fechas'] = l
test_2

entregable = pd.DataFrame(columns=['Fecha', 'Velocidad'])

j=0

for i in test_2['Fechas']:
  entregable.loc[j] = [i, predicciones[predicciones['Fecha']==invertir(i)]['Velocidad'].values[0]]
  j+=1

In [ ]:
entregable

,Fecha,Velocidad
0,26-04-2015 01:50,7.766310
1,26-04-2015 03:10,6.516062
2,26-04-2015 04:20,6.677339
3,26-04-2015 05:20,6.389441
4,26-04-2015 06:40,6.459248
...,...,...
1995,19-09-2015 08:10,8.960595
1996,21-09-2015 11:20,7.621277
1997,22-09-2015 10:30,7.842158
1998,23-09-2015 17:30,11.589312


In [ ]:
entregable = vector(entregable)
entregable.to_csv('sampleSubmission.csv')

# Luong attention

In [ ]:
def luong_model(X_input_train, X_output_train):

  n_hidden = 100
  input_train = Input(shape=(X_input_train.shape[1], 1)) #N_features
  output_train = Input(shape=(X_output_train.shape[1], 1))

  #Encoder

  encoder_stack_h, encoder_last_h, encoder_last_c = LSTM(
    n_hidden, activation='elu', dropout=0.2, recurrent_dropout=0.2, 
    return_state=True, return_sequences=True)(input_train)

  #Normalization

  encoder_last_h = BatchNormalization(momentum=0.6)(encoder_last_h)
  encoder_last_c = BatchNormalization(momentum=0.6)(encoder_last_c)  

  #Decoder
  
  decoder_input = RepeatVector(output_train.shape[1])(encoder_last_h)

  decoder_stack_h = LSTM(n_hidden, activation='elu', dropout=0.2, recurrent_dropout=0.2,
                          return_state=False, return_sequences=True)(
                          decoder_input, initial_state=[encoder_last_h, encoder_last_c])

  #Attention layer
  
  attention = dot([decoder_stack_h, encoder_stack_h], axes=[2, 2])
  attention = Activation('softmax')(attention)

  #Context vector + Normalization

  context = dot([attention, encoder_stack_h], axes=[2,1])
  #context = BatchNormalization(momentum=0.6)(context)

  #Stack vectors 

  decoder_combined_context = concatenate([context, decoder_stack_h])

  out = TimeDistributed(Dense(output_train.shape[2]))(decoder_combined_context)

  return out, input_train, output_train

In [ ]:
def predicciones_luong(fecha_inf_2013, fecha_sup_2013, fecha_inf_2014, fecha_sup_2014):

  df_2013 = train[(train['Fecha']<= fecha_sup_2013)&(fecha_inf_2013 <= train['Fecha'])]
  df_2014 = train[(train['Fecha']<= fecha_sup_2014)&(fecha_inf_2014 <= train['Fecha'])]

  X, _ = preprocesamiento(np.array(df_2013.Velocidad), 7, 4, 3)
  _, y = preprocesamiento(np.array(df_2013.Velocidad), 7, 4, 3)

  out, input_train, output_train= luong_model(X,y)

  model = Model(inputs=input_train, outputs=out)
  opt = Adam(lr=0.01, clipnorm=1)
  model.compile(loss='mean_squared_error', optimizer=opt, metrics=['mae'])
  
  X2, y2 = preprocesamiento(np.array(df_2014.Velocidad), 7, 4, 3)

  y_pred = model.predict(X2)

  pred = df_predicciones(df_2014, y_pred)

  return pred

In [ ]:
def df_predicciones_luong(df, y_pred):

  pred = pd.DataFrame(columns=['Fecha','Velocidad'])

  pred['Fecha'] = list(df['Fecha']+ timedelta(days=365))[7:-2]
  pred['Velocidad'] = y_pred

  return pred

In [ ]:
df_2013 = train[(train['Fecha']<= '2013-05-01')&('2013-03-27' <= train['Fecha'])]
df_2014 = train[(train['Fecha']<= '2014-05-01')&('2014-03-27' <= train['Fecha'])]

X, _ = preprocesamiento(np.array(df_2013.Velocidad), 7, 3, 3)
_, y = preprocesamiento(np.array(df_2013.Velocidad), 7, 3, 3)
out, input_train, output_train= luong_model(X,y)

model = Model(inputs=input_train, outputs=out)
opt = Adam(lr=0.01, clipnorm=1)
model.compile(loss='mean_squared_error', optimizer=opt, metrics=['mae'])
  
X2, y2 = preprocesamiento(np.array(df_2014.Velocidad), 7, 3, 3)

y_pred = model.predict(X2)

In [ ]:
y_pred

array([[[-0.8263902 ],
        [-0.83522886],
        [-0.84690505]],

       [[-0.68974584],
        [-0.6883785 ],
        [-0.6924424 ]],

       [[-0.8571954 ],
        [-0.8390426 ],
        [-0.8271793 ]],

       ...,

       [[-1.4347496 ],
        [-1.4006119 ],
        [-1.3914354 ]],

       [[-1.728497  ],
        [-1.6565052 ],
        [-1.6370885 ]],

       [[-1.8636352 ],
        [-1.7905161 ],
        [-1.7721868 ]]], dtype=float32)